https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1


https://towardsdatascience.com/similarity-metrics-in-nlp-acc0777e234c 

https://www.youtube.com/watch?v=Ey81KfQ3PQU


Model semantic search: 'multi-qa-MiniLM-L6-cos-v1'

Model assymetric : 'msmarco-MiniLM-L-6-v3'

In [ ]:
import pandas as pd
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
from Data_with_NLP import *
import pandas as pd
# import pandasql as ps
import matplotlib.pyplot as plt
# Import argsort
from numpy import argsort
from Functions.connection.connection import *
from Functions.AlertP1.data_cleaning import *
from Functions.AlertP1.features import *
from Functions.analysis.step_analysis import *
from Functions.AlertP1.dummy_features import *
from Functions.Models.Logistic_regression import *
from Functions.Models.evaluation import *
from Data_with_NLP import *
from Functions.pipeline import *

: 

In [ ]:
protocols = pd.read_csv('/Users/gabrielabib/Desktop/NOVA_SBE/PBL/BERT/protocols.csv')
protocols_acceptance = protocols[['Acceptance']]
protocols_refusal = protocols[['Refusal']]
protocols_acceptance.dropna(inplace=True)

: 

In [ ]:
# # 1. Data cleaning
lower_text(protocols_acceptance,'Acceptance', 'Acceptance')
lower_text(protocols_refusal,'Refusal', 'Refusal')
remove_stop_words(protocols_acceptance,'Acceptance', 'Acceptance')
remove_stop_words(protocols_refusal,'Refusal', 'Refusal')
spacy_lemmatizer(protocols_acceptance,'Acceptance', 'Acceptance')
spacy_lemmatizer(protocols_refusal,'Refusal', 'Refusal')

protocols_acceptance

: 

In [ ]:
acceptance_list = protocols_acceptance['Acceptance'].tolist()
acceptance_list

: 

In [ ]:
[acceptance_list[0]]

: 

In [ ]:
alertP1 = connection("credentials.csv")
alertP1 = alertP1[alertP1['text_length']>0]


: 

In [ ]:
alertP1_simplified = alertP1[['COD_REFERENCIA', 'Texto']]

: 

In [ ]:
alertP1_simplified.info()

: 

In [ ]:
lower_text(alertP1_simplified,'Texto', 'Texto')
remove_stop_words(alertP1_simplified,'Texto', 'Texto_Stop_Words')
spacy_lemmatizer(alertP1_simplified,'Texto_Stop_Words', 'text_lemmatized')
alertP1_simplified

: 

In [ ]:
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
import torch


: 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

: 

In [ ]:
# # THIS SHOULD BE DELETED

# alertP1_simplified = alertP1_simplified[:100]
# alertP1_simplified

: 

In [ ]:
# referrals = alertP1_simplified['text_lemmatized'].tolist()


# for index, protocol in enumerate(acceptance_list):

#     sentences = [protocol] + referrals

#     # the following line is only necessary if you want to limit the number of sentences. MUST BE COMMENTED OUT TO RUN ON FULL DATASET
#     # sentences = sentences[:100]

#     # initialize dictionary to store tokenized sentences
#     tokens = {'input_ids': [], 'attention_mask': []}

#     for sentence in sentences:
#         # encode each sentence and append to dictionary
#         new_tokens = tokenizer.encode_plus(sentence, max_length=128,
#                                         truncation=True, padding='max_length',
#                                         return_tensors='pt')
#         tokens['input_ids'].append(new_tokens['input_ids'][0])
#         tokens['attention_mask'].append(new_tokens['attention_mask'][0])

#     # reformat list of tensors into single tensor
#     tokens['input_ids'] = torch.stack(tokens['input_ids'])
#     tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
        
#     outputs = model(**tokens)
#     outputs.keys()

#     embeddings = outputs.last_hidden_state
#     # embeddings

#     attention_mask = tokens['attention_mask']
#     # attention_mask.shape
#     mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
#     # mask.shape

#     masked_embeddings = embeddings * mask
#     # masked_embeddings.shape

#     summed = torch.sum(masked_embeddings, 1)
#     # summed.shape

#     summed_mask = torch.clamp(mask.sum(1), min=1e-9)
#     summed_mask.shape

#     mean_pooled = summed / summed_mask

#     # convert from PyTorch tensor to numpy array
#     mean_pooled = mean_pooled.detach().numpy()

#     # calculate
#     results = cosine_similarity(
#         [mean_pooled[0]],
#         mean_pooled[1:]
#     )

#     alertP1_simplified[index] = results[0]



: 

In [ ]:
referrals = alertP1_simplified['text_lemmatized'].tolist()


protocol = acceptance_list[0]

sentences = [protocol] + referrals

# the following line is only necessary if you want to limit the number of sentences. MUST BE COMMENTED OUT TO RUN ON FULL DATASET
# sentences = sentences[:100]

# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                    truncation=True, padding='max_length',
                                    return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])
    
outputs = model(**tokens)
outputs.keys()

embeddings = outputs.last_hidden_state
# embeddings

attention_mask = tokens['attention_mask']
# attention_mask.shape
mask = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
# mask.shape

masked_embeddings = embeddings * mask
# masked_embeddings.shape

summed = torch.sum(masked_embeddings, 1)
# summed.shape

summed_mask = torch.clamp(mask.sum(1), min=1e-9)
summed_mask.shape

mean_pooled = summed / summed_mask

# convert from PyTorch tensor to numpy array
mean_pooled = mean_pooled.detach().numpy()

# calculate
results = cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

alertP1_simplified['0'] = results[0]




: 

In [ ]:
sentences

: 

: 